# Signal to the model isn't allowing for improved experimentation results. I'm testing the possibility of improvement through the application of feature engineering. This is counter intuitive to what I've learnd in the past about deep neural networks beause they offer a natrual feature engineering behavior thorugh layers, nodes and activation functions. 

In [43]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import BinaryFocalCrossentropy
import pandas as pd
from sklearn.metrics import (
    f1_score, precision_score, recall_score, roc_auc_score, average_precision_score,
    classification_report, multilabel_confusion_matrix,precision_recall_curve
)
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit as MLSSS
import matplotlib.pyplot as plt

import optuna
from optuna.visualization import (
    plot_optimization_history, plot_param_importances, plot_slice
)
import matplotlib.pyplot as plt
from pyprojroot import here
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report


In [44]:
multi_label_data = pd.read_csv(here('data/processed/joint_target_vectors_df.csv'))
multi_label_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   quality_variation        10000 non-null  int64  
 1   Air temperature [K]      10000 non-null  float64
 2   Process temperature [K]  10000 non-null  float64
 3   Rotational speed [rpm]   10000 non-null  float64
 4   Torque [Nm]              10000 non-null  float64
 5   Tool wear [min]          10000 non-null  float64
 6   TWF                      10000 non-null  int64  
 7   HDF                      10000 non-null  int64  
 8   PWF                      10000 non-null  int64  
 9   OSF                      10000 non-null  int64  
 10  RNF                      10000 non-null  int64  
 11  joint_PWF_OSF            10000 non-null  int64  
 12  joint_TWF_RNF            10000 non-null  int64  
 13  joint_HDF_PWF            10000 non-null  int64  
 14  joint_HDF_OSF          

In [45]:
targets = ['TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'joint_PWF_OSF',
            'joint_TWF_RNF', 'joint_HDF_PWF', 'joint_HDF_OSF',
            'joint_TWF_OSF', 'joint_TWF_PWF_OSF']

In [46]:
multi_label_data['Toolwear*Torque'] = multi_label_data['Tool wear [min]'] * multi_label_data['Torque [Nm]']

In [47]:
multi_label_data['Toolwear-Strain'] = multi_label_data['Process temperature [K]'] - multi_label_data['Air temperature [K]']

In [48]:
multi_label_data['TorquetimesInvRPM'] = multi_label_data['Torque [Nm]'] * (1 / multi_label_data['Rotational speed [rpm]'])

In [51]:
multi_label_data['Toolwear*AirTemp'] = multi_label_data['Tool wear [min]'] * (multi_label_data['Air temperature [K]'])



In [52]:
multi_label_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   quality_variation        10000 non-null  int64  
 1   Air temperature [K]      10000 non-null  float64
 2   Process temperature [K]  10000 non-null  float64
 3   Rotational speed [rpm]   10000 non-null  float64
 4   Torque [Nm]              10000 non-null  float64
 5   Tool wear [min]          10000 non-null  float64
 6   TWF                      10000 non-null  int64  
 7   HDF                      10000 non-null  int64  
 8   PWF                      10000 non-null  int64  
 9   OSF                      10000 non-null  int64  
 10  RNF                      10000 non-null  int64  
 11  joint_PWF_OSF            10000 non-null  int64  
 12  joint_TWF_RNF            10000 non-null  int64  
 13  joint_HDF_PWF            10000 non-null  int64  
 14  joint_HDF_OSF          

In [53]:
multi_label_data.head()

,quality_variation,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,...,joint_PWF_OSF,joint_TWF_RNF,joint_HDF_PWF,joint_HDF_OSF,joint_TWF_OSF,joint_TWF_PWF_OSF,Toolwear*Torque,Toolwear-Strain,TorquetimesInvRPM,Toolwear*AirTemp
0,1,-0.952389,-0.947360,0.068185,0.282200,-1.695984,0,0,0,0,...,0,0,0,0,0,0,-0.478606,0.005030,4.138728,1.615237
1,0,-0.902393,-0.879959,-0.729472,0.633308,-1.648852,0,0,0,0,...,0,0,0,0,0,0,-1.044231,0.022434,-0.868174,1.487913
2,0,-0.952389,-1.014761,-0.227450,0.944290,-1.617430,0,0,0,0,...,0,0,0,0,0,0,-1.527323,-0.062371,-4.151639,1.540424
3,0,-0.902393,-0.947360,-0.590021,-0.048845,-1.586009,0,0,0,0,...,0,0,0,0,0,0,0.077469,-0.044966,0.082785,1.431204
4,0,-0.902393,-0.879959,-0.729472,0.001313,-1.554588,0,0,0,0,...,0,0,0,0,0,0,-0.002041,0.022434,-0.001800,1.402850


In [54]:
best_params = {'n_layers': 6, 'gamma': 0.9003549934636083, 'units_l0': 577, 'units_l1': 184, 'units_l2': 214, 'units_l3': 206, 'units_l4': 801, 'lr': 2.8746290296601193e-05, 'epochs': 97, 'beta_1': 0.8963081622002852, 'beta_2': 0.9366159181660023, 'epsilon': 8.562431439031271e-07, 'alpha_pow': 0.9534631280825616}


In [55]:
# ---- Quick train/val split (stratify on "any failure" just to be sane) ----
y_any = (multi_label_data[targets].sum(axis=1) > 0).astype(int)

train_df, val_df = train_test_split(
    multi_label_data,
    test_size=0.2,
    random_state=23,
    stratify=y_any
)

# ---- Scale numeric features (train stats only) ----
num_cols = [
    'Tool wear [min]',
    'Torque [Nm]',
    'Rotational speed [rpm]',
    'Air temperature [K]',
    'Process temperature [K]',
    'Toolwear*Torque',
    'Toolwear-Strain',
    'TorquetimesInvRPM',
    'Toolwear*AirTemp',
]


train_df[num_cols] = train_df[num_cols]
val_df[num_cols]   = val_df[num_cols]

y_train = train_df[targets].values.astype('float64')
y_val   = val_df[targets].values.astype('float64')

In [60]:
def build_model(hidden_layer_sizes=None, bias_init_value=None):
    if hidden_layer_sizes is None:
        hidden_layer_sizes = []

    tf.keras.backend.clear_session()
    tf.random.set_seed(23)

    # --- Inputs ---
    toolWear = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Tool wear [min]')
    torque   = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Torque [Nm]')
    rotationalSpeed = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Rotational speed [rpm]')
    qualityVariation = tf.keras.layers.Input(shape=(1,), dtype=tf.int64,  name='quality_variation')
    airTemperature = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Air temperature [K]')
    processTemperature = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Process temperature [K]')

    # NEW engineered-feature inputs
    toolwear_torque  = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Toolwear*Torque')
    toolwear_strain  = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Toolwear-Strain')
    torque_inv_rpm   = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='TorquetimesInvRPM')
    toolwear_airtemp = tf.keras.layers.Input(shape=(1,), dtype=tf.float64, name='Toolwear*AirTemp')

    # Cast qualityVariation to float for Dense layers
    qualityVariation_float = tf.keras.layers.Lambda(
        lambda t: tf.cast(t, tf.float64),
        name="quality_variation_to_float"
    )(qualityVariation)

    # Concatenate all features
    x = tf.keras.layers.Concatenate(name="feature_concat")([
        toolWear,
        torque,
        rotationalSpeed,
        qualityVariation_float,
        airTemperature,
        processTemperature,
        toolwear_torque,
        toolwear_strain,
        torque_inv_rpm,
        toolwear_airtemp,
    ])

    # Hidden layers
    for units in hidden_layer_sizes:
        x = tf.keras.layers.Dense(units, activation='relu', dtype=tf.float64)(x)

    # Bias initialization (we'll just use 0 here for this quick test)
    if bias_init_value is not None:
        output_bias = tf.keras.initializers.Constant(bias_init_value)
    else:
        output_bias = tf.keras.initializers.Constant(0.0)

    outputs = tf.keras.layers.Dense(
        len(targets),
        activation='sigmoid',
        bias_initializer=output_bias,
        dtype=tf.float64
    )(x)

    model = tf.keras.Model(
        inputs=[
            toolWear, torque, rotationalSpeed,
            qualityVariation, airTemperature, processTemperature,
            toolwear_torque, toolwear_strain, torque_inv_rpm, toolwear_airtemp
        ],
        outputs=outputs,
        name="multilabel_failure_model",
    )

    return model


def to_input_list(df):
    """Helper: convert df to list of model inputs in the correct order."""
    return [
        df['Tool wear [min]'].values,
        df['Torque [Nm]'].values,
        df['Rotational speed [rpm]'].values,
        df['quality_variation'].values,
        df['Air temperature [K]'].values,
        df['Process temperature [K]'].values,
        df['Toolwear*Torque'].values,
        df['Toolwear-Strain'].values,
        df['TorquetimesInvRPM'].values,
        df['Toolwear*AirTemp'].values,
    ]


In [66]:
# --- Build model as you already did, using engineered features ---
hidden_layer_sizes = [
    best_params['units_l0'],
    best_params['units_l1'],
    best_params['units_l2'],
    best_params['units_l3'],
    best_params['units_l4'],
]
model = build_model(hidden_layer_sizes=hidden_layer_sizes)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=best_params['lr'],
    beta_1=best_params['beta_1'],
    beta_2=best_params['beta_2'],
    epsilon=best_params['epsilon'],
)

# >>> Binary Focal Loss (simple version) <<<
bfl_loss = tf.keras.losses.BinaryFocalCrossentropy(
    gamma=15.0,
    alpha=.96,
    apply_class_balancing=True,
    from_logits=False,
    reduction="sum_over_batch_size"
)

model.compile(
    optimizer=optimizer,
    loss=bfl_loss,
    metrics=[]   # we'll compute F1 with sklearn like before
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    mode='min'
)

history = model.fit(
    to_input_list(train_df),
    y_train,
    validation_data=(to_input_list(val_df), y_val),
    epochs=int(best_params['epochs']),
    batch_size=8,
    callbacks=[es],
    verbose=1,
)


from sklearn.metrics import f1_score, classification_report

val_probs = model.predict(to_input_list(val_df), verbose=0)
val_pred  = (val_probs >= 0.5).astype(int)

macro_f1 = f1_score(y_val, val_pred, average='macro', zero_division=0)
micro_f1 = f1_score(y_val, val_pred, average='micro', zero_division=0)

print("\nValidation focal loss (last epoch):", history.history['val_loss'][-1])
print("Macro F1 @0.5:", macro_f1)
print("Micro F1 @0.5:", micro_f1)

print("\nPer-label classification report:\n")
print(classification_report(y_val, val_pred, target_names=targets, zero_division=0))



Epoch 1/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 6.8047e-07 - val_loss: 6.2692e-07
Epoch 2/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 5.8054e-07 - val_loss: 5.4761e-07
Epoch 3/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 5.1800e-07 - val_loss: 4.9501e-07
Epoch 4/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 4.7586e-07 - val_loss: 4.5775e-07
Epoch 5/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 4.4567e-07 - val_loss: 4.2994e-07
Epoch 6/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 4.2276e-07 - val_loss: 4.0816e-07
Epoch 7/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 4.0446e-07 - val_loss: 3.9051e-07
Epoch 8/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 3.8929e-07 - val_loss: 3.7569e-07
Epoch 9/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 3.7630e-07 - val_loss: 3.6298e-07
Epoch 10/97
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 3.6492e-07 - val_loss: 3.5190e-07
Epoch 11/97
1000/1000 ━━━━━━━━━━━━━━━━━

# Feature engineering is showing signes of improvement in performance. Transferring to experimentation space for optuna